In [1]:
import sys
import os
sys.path.append(os.path.join("..", ".."))
import model
import model.sim.io
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sim = model.sim.io.load_pickle()

In [3]:
import torch
import numpy as np

N = sim['patients'].shape[0]
data = sim['contacts'][['patient1', 'patient2']].to_numpy()
def get_batch(
        batch_size = 32,
        negative_sample = 0.5):
    negative_sample = int(batch_size * negative_sample)
    inputs = np.concatenate([
        data[np.random.choice(data.shape[0], batch_size-negative_sample)],
        np.random.choice(N, negative_sample*2, replace=True).reshape(-1, 2)]).astype(np.int64)
    targets = np.array([0] * negative_sample + [1] * (batch_size - negative_sample)).astype(np.float64)
    return torch.tensor(inputs[:,0]), torch.tensor(inputs[:,1]), torch.tensor(targets)


embedding = torch.nn.Embedding(N, embedding_dim=256)
lr = 1e-3
optimizer = torch.optim.Adam(embedding.parameters(), lr=lr)

batch_size = 1024
n_epochs = 50
print_every = 1
device = torch.device('cuda')
embedding = embedding.to(device)
n_batches = sim['contacts'].shape[0] // batch_size
for i in range(n_epochs):
    try:
        batch_loss = []
        for _ in range(n_batches):
            query_patients, contact_patients, targets = get_batch()
            query_patients = query_patients.to(device)
            contact_patients = contact_patients.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            loss = torch.binary_cross_entropy_with_logits(torch.diag(torch.matmul(embedding(query_patients), embedding(contact_patients).T)), targets)
            loss = torch.sum(loss)
            batch_loss.append(loss.item())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(embedding.parameters(), 0.5)
            optimizer.step()
        if i % print_every == 0:
            print("Epoch {}: loss = {:.3f}".format(i, np.mean(batch_loss)))
    except KeyboardInterrupt:
        break

Epoch 0: loss = 204.379
Epoch 1: loss = 200.531
Epoch 2: loss = 196.072
Epoch 3: loss = 192.259
Epoch 4: loss = 183.441
Epoch 5: loss = 183.786
Epoch 6: loss = 175.568
Epoch 7: loss = 172.819
Epoch 8: loss = 169.161
Epoch 9: loss = 167.914
Epoch 10: loss = 164.513
Epoch 11: loss = 164.181
Epoch 12: loss = 157.645
Epoch 13: loss = 156.864
Epoch 14: loss = 152.474
Epoch 15: loss = 148.004
Epoch 16: loss = 146.979
Epoch 17: loss = 143.065
Epoch 18: loss = 140.465
Epoch 19: loss = 137.032
Epoch 20: loss = 137.824
Epoch 21: loss = 135.214
Epoch 22: loss = 128.719
Epoch 23: loss = 128.261
Epoch 24: loss = 124.548
Epoch 25: loss = 125.079
Epoch 26: loss = 123.109
Epoch 27: loss = 121.665
Epoch 28: loss = 118.667
Epoch 29: loss = 117.204
Epoch 30: loss = 115.559
Epoch 31: loss = 114.851
Epoch 32: loss = 111.458
Epoch 33: loss = 109.139
Epoch 34: loss = 105.910
Epoch 35: loss = 107.923
Epoch 36: loss = 106.888
Epoch 37: loss = 106.972
Epoch 38: loss = 101.926
Epoch 39: loss = 100.301
Epoch 40: 

In [4]:
# 0: unknown
# 1: test negative
# 2: test positive
# 3: hospitalized
# 4: dead
known_state = np.zeros((sim['patients'].shape[0], sim['dates'].shape[0]), dtype=int)
for date in sim['dates']['date']:
    tests_df = sim['tests'].loc[sim['tests']['date'] == date]
    known_state[tests_df.loc[np.logical_not(tests_df['result'])]['patient'].astype(int), date:] = 1
    known_state[tests_df.loc[tests_df['result']]['patient'].astype(int), date:] = 2
    known_state[sim['hospital'].loc[sim['hospital']['date'] == date,'patient'].astype(int), date:] = 3
    known_state[sim['deaths'].loc[sim['deaths']['date'] == date,'patient'].astype(int), date:] = 4

targets = np.zeros((sim['patients'].shape[0], 5, sim['dates'].shape[0]))
for date in sim['dates']['date']:
    targets[(np.arange(sim['patients'].shape[0]),known_state[:,date], np.repeat(date, sim['patients'].shape[0]))] = 1

In [5]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


class DynamicEmbedding(nn.Module):
    
    def __init__(self, embedding, n_timepoints):
        super().__init__()
        self.n_timepoints = n_timepoints
        self.num_embeddings = embedding.num_embeddings
        self.embedding_dim = embedding.embedding_dim
        self.embedding = nn.Embedding(
            self.num_embeddings * self.n_timepoints, self.embedding_dim, 
            _weight=torch.stack([embedding.weight
                                 for _ in range(n_timepoints)]).reshape(-1, self.embedding_dim))
        all = torch.tensor(np.arange(self.num_embeddings, dtype=np.int64)).to(self.embedding.weight.device)
        self.register_buffer('all', all)
    
    def forward(self, x, t):
        x_masked = torch.where(x < 0, torch.tensor(0).to(x.device), x)
        t = torch.tensor(t*self.num_embeddings).unsqueeze(1)
        x_offset = x_masked + t.to(x.device)
        x_embedded = self.embedding.forward(x_offset)
        x_embedded_masked = torch.where(x.unsqueeze(-1) < 0, torch.zeros(*x.size(), 1).to(x.device), x_embedded)
        return x_embedded_masked
    
    def loss(self):
        loss = 0
        prev_timepoint = self.forward(self.all, 0)
        for t in range(1, self.n_timepoints):
            curr_timepoint = self.forward(self.all, t)
            loss += torch.sum(1 - torch.nn.functional.cosine_similarity(curr_timepoint, prev_timepoint))
            prev_timepoint = curr_timepoint
        return loss

In [6]:

class TransformerModel(nn.Module):

    def __init__(self, embedding, n_timepoints, n_outputs, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.embedding_dim = embedding.embedding_dim
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(self.embedding_dim, dropout)
        encoder_layers = TransformerEncoderLayer(self.embedding_dim, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.embedding = embedding
        self.decoder = nn.Linear(self.embedding_dim, n_outputs)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, t, positions=None):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.embedding(src, t) * math.sqrt(self.embedding_dim)
        src = self.pos_encoder(src, positions)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.max_len = max_len
        self.input_size = d_model
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)).unsqueeze(0).unsqueeze(0)
        self.register_buffer('div_term', div_term)
    
    def embed(self, positions, x):
        if positions is None:
            positions = torch.arange(0, x.size(1), dtype=torch.float).to(x.device).unsqueeze(0)
        n = positions.size(1)
        pe = torch.zeros(x.size(0), n, self.input_size).to(x.device)
        sin_embedding = torch.sin(positions.unsqueeze(2) * self.div_term)
        cos_embedding = torch.cos(positions.unsqueeze(2) * self.div_term)
        pe[:, :, 0::2] = sin_embedding
        pe[:, :, 1::2] = cos_embedding
        return pe

    def forward(self, x, positions):
        x = x + self.embed(positions, x=x)
        return self.dropout(x)

In [7]:
class DiseaseNet(nn.Module):
    def __init__(self, embedding, n_timepoints, n_outputs, 
                 transformer_n_head=2, transformer_n_hidden=256, transformer_n_layers=2, 
                 ff_n_hidden=256, ff_n_layers=2,
                 dropout=0.5):
        super().__init__()
        self.embedding = DynamicEmbedding(embedding, n_timepoints)
        self.transformer = TransformerModel(self.embedding, n_timepoints, n_outputs=embedding.embedding_dim, 
                         nhead=transformer_n_head, nhid=transformer_n_hidden, nlayers=transformer_n_layers, dropout=dropout)
        input_dim = embedding.embedding_dim * 2
        hidden_layers = []
        for _ in range(ff_n_layers):
            hidden_layers += [
                nn.Linear(input_dim, ff_n_hidden),
                nn.LeakyReLU(),
                nn.Dropout(dropout),
            ]
            input_dim = ff_n_hidden
        self.decoder = nn.Sequential(
            *hidden_layers,
            nn.Linear(input_dim, n_outputs))
        
        pos_weight = torch.ones([n_outputs])
        pos_weight[0] = 0
        self.loss = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    def forward(self, x, t, contacts, positions=None):
        x_embed = self.embedding(x, t)
        contacts_embed = self.transformer(contacts, t, positions)[:,-1]
        output = self.decoder(torch.cat((x_embed.squeeze(1), contacts_embed), 1))
        return output

In [8]:
model = DiseaseNet(embedding, sim['dates'].shape[0], n_outputs=targets.shape[1]).to(device)

In [9]:
def pad_sequence(sequences, batch_first=False, padding_value=0):
    max_size = sequences[0].size()
    trailing_dims = max_size[1:]
    max_len = max([s.size(0) for s in sequences])
    if batch_first:
        out_dims = (len(sequences), max_len) + trailing_dims
    else:
        out_dims = (max_len, len(sequences)) + trailing_dims

    out_tensor = sequences[0].data.new(*out_dims).fill_(padding_value)
    for i, tensor in enumerate(sequences):
        length = tensor.size(0)
        # use index notation to prevent duplicate references to the tensor
        if batch_first:
            out_tensor[i, -length:, ...] = tensor
        else:
            out_tensor[-length:, i, ...] = tensor

    return out_tensor

def get_contacts(p, t):
    contacts_df = sim['contacts'].loc[(sim['contacts']['date'] <= t) & (sim['contacts']['date'] > t-15)]
    contacts = np.concatenate([
        contacts_df.loc[contacts_df['patient1'] == p]['patient2'].to_numpy(),
        contacts_df.loc[contacts_df['patient2'] == p]['patient1'].to_numpy()
    ])
    dates = np.concatenate([
        contacts_df.loc[contacts_df['patient1'] == p]['date'].to_numpy(),
        contacts_df.loc[contacts_df['patient2'] == p]['date'].to_numpy()
    ])
    assert len(contacts) == len(dates)
    return torch.tensor(contacts), torch.tensor(dates)

batch_size=32


def get_batch(batch_size=128):
    p = np.random.choice(np.arange(sim['patients'].shape[0], dtype=np.int64), batch_size, replace=True)
    t = np.random.choice(sim['dates'].shape[0], batch_size, replace=True)
    contacts, dates = zip(*[get_contacts(patient, time) for patient, time in zip(p,t)])
    if np.min([len(c) for c in contacts]) == 0:
        return get_batch(batch_size)
    max_len = np.max([len(c) for c in contacts])
    contacts, dates = pad_sequence(contacts, True, -1), pad_sequence(dates, True, 0)
    target = torch.tensor(targets[p,:,t].astype(np.float64))
    p = torch.tensor(p).unsqueeze(1)
    return p, t, contacts, dates, target

In [11]:
lr = 1e-3 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train(batch_size=256):
    model.train() # Turn on the train mode
    n_batch = sim['patients'].shape[0] * sim['dates'].shape[0] // batch_size
    curr_loss = 0.
    total_loss = 0.
    start_time = time.time()
    for i in range(n_batch):
        p, t, contacts, dates, target = get_batch(batch_size=batch_size)
        if contacts.size(1) == 0:
            continue
        optimizer.zero_grad()
        output = model(p.to(device), t, contacts.to(device), dates.to(device))
        loss = model.loss(output, target.to(device))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        curr_loss += loss.item()
        total_loss += loss.item()
        log_interval = 200
        if i % log_interval == 0 and i > 0:
            curr_loss = curr_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, i, n_batch, #scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    curr_loss, np.exp(curr_loss)))
            curr_loss = 0.
            start_time = time.time()
    return total_loss / n_batch

In [13]:
best_val_loss = float("inf")
epochs = 20 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    val_loss = train()
#     val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, np.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    #scheduler.step()

| epoch   1 |   200/  781 batches | ms/batch 1919.89 | loss  0.04 | ppl     1.05
| epoch   1 |   400/  781 batches | ms/batch 1753.89 | loss  0.04 | ppl     1.04
| epoch   1 |   600/  781 batches | ms/batch 1711.88 | loss  0.03 | ppl     1.03
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 1383.63s | valid loss  0.04 | valid ppl     1.04
-----------------------------------------------------------------------------------------
| epoch   2 |   200/  781 batches | ms/batch 1723.53 | loss  0.02 | ppl     1.02
| epoch   2 |   400/  781 batches | ms/batch 1707.94 | loss  0.02 | ppl     1.02
| epoch   2 |   600/  781 batches | ms/batch 1690.91 | loss  0.02 | ppl     1.02
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 1326.37s | valid loss  0.02 | valid ppl     1.02
-----------------------------------------------------------------------------------------
| 

-----------------------------------------------------------------------------------------
| end of epoch  17 | time: 1691.36s | valid loss  0.00 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch  18 |   200/  781 batches | ms/batch 2208.32 | loss  0.00 | ppl     1.00
| epoch  18 |   400/  781 batches | ms/batch 2128.31 | loss  0.00 | ppl     1.00
| epoch  18 |   600/  781 batches | ms/batch 2101.57 | loss  0.00 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  18 | time: 1655.96s | valid loss  0.00 | valid ppl     1.00
-----------------------------------------------------------------------------------------
| epoch  19 |   200/  781 batches | ms/batch 1878.11 | loss  0.00 | ppl     1.00
| epoch  19 |   400/  781 batches | ms/batch 2131.71 | loss  0.00 | ppl     1.00
| epoch  19 |   600/  781 batches | ms/batch 2106.08 | loss  0.00 | ppl     1.00
--

In [ ]:
test_loss = evaluate(best_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)